In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

landuse = gpd.read_file("L3_TAXPAR_POLY_ASSESS_gdb_3615755504676150852/masslanduse.geojson")

In [3]:
station_areas = pd.read_csv("../stations_routes_f23_geoms_nonoverlapping.csv")

station_areas= gpd.GeoDataFrame(station_areas,geometry=gpd.GeoSeries.from_wkt(station_areas["walk_poly_wkt"]))
station_areas = station_areas.set_crs("EPSG:4326").to_crs("EPSG:26986")

In [25]:
intersecting_parcels = landuse.sjoin(station_areas,predicate="intersects")[["MAP_PAR_ID","SITE_ADDR","PROP_ID","USE_CODE","ZONING","BLD_AREA","RES_AREA","LOT_SIZE","route_id","parent_station","geometry"]].reset_index().drop(columns=["index"]).dropna(subset=["SITE_ADDR"])

In [26]:
landuse_dict = {"Mixed Use":0,"Residential":1,"Commercial":3,"Industrial":4,"Other":[2,5,6,7,8,9]}
mapping = {}
for key, value in landuse_dict.items():
    if isinstance(value, list):
        for v in value:
            mapping[v] = key
    else:
        mapping[value] = key

# Function to classify based on the first digit
def classify_land_use(value):
    first_digit = int(str(value)[0])  # Get the first digit
    return mapping.get(first_digit, "Unknown")  # Default to "Unknown" if not found

# Apply the classification to the DataFrame
intersecting_parcels['classification'] = intersecting_parcels['USE_CODE'].apply(lambda x: classify_land_use(x))

In [ ]:
#categorize use
#percent area-overlapping parcels get even split of lot size
#cut to walksheds
#calculate entropy index for each walkshed

In [34]:
# groupby station area
# sum of building area
# each parcel living area divided by sum
# groupby classification
# entropy index for each station area

total_areas = intersecting_parcels.groupby(["route_id","parent_station"]).sum()["BLD_AREA"].reset_index().rename(columns={"BLD_AREA":"total_area"})

intersecting_parcels=intersecting_parcels.merge(total_areas[["route_id","parent_station","total_area"]],on=["route_id","parent_station"])

In [36]:
intersecting_parcels["pct_total"] = intersecting_parcels["BLD_AREA"] / intersecting_parcels["total_area"]

In [40]:
intersecting_parcels=intersecting_parcels.groupby(["route_id","parent_station","classification"]).sum()["pct_total"].reset_index()

In [50]:
import numpy as np

def calculate_entropy(group):
    k = 5
    entropy = -np.sum(group['pct_total'] * np.log(group['pct_total'])) / np.log(k)
    return entropy


lu_entropy=intersecting_parcels.groupby(["route_id","parent_station"]).apply(lambda x: calculate_entropy(x)).reset_index().rename(columns={0:"land_use_entropy"})

c:\Users\Ted Banken\.conda\envs\sal\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [52]:
lu_entropy.to_csv("station_routes_2024_lu_entropy.csv")